In [157]:
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd

index_list = ['Product Name', 'Product Quanity Available', 'Propuct Instock',
 'Product Listing number', 'Product Year', 'Metal Type',
 'Metal Content', 'Oz weight', 'Finess', 'Gold Value', 'Price per gram', 'Price 1', 'Price 2', 'Price 5',
 'Price 10', 'Price 20','Price 50', 'Price 100', 'Product link'
]

df = pd.DataFrame(columns=index_list)
oz = 28.35

# Royal Mint

In [158]:
royal_mint = r'https://www.royalmint.com/invest/bullion/bullion-coins/gold-coins/'
royal_mint_base_url = r'https://www.royalmint.com'

royal_mint_soup = BeautifulSoup(requests.get(royal_mint).content, 'html.parser')
royal_mint_full_list = royal_mint_soup.find_all(class_='product-card product-consumer analytic-data-attribute')

gold_price = float(royal_mint_soup.find('a', 'gold').find('span', 'value').text.replace(',', '')[1:])
silver_price = float(royal_mint_soup.find('a', 'silver').find('span', 'value').text[1:])
platinum_price = float(royal_mint_soup.find('a', 'platinum').find('span', 'value').text[1:])

royal_mint_soup = None

In [164]:
for product in royal_mint_full_list:
    product_name = product.find('h3', 'product-name').find('a')['title']
    product_href = product.find('h3', 'product-name').find('a')['href']
    product_href = royal_mint_base_url + product_href
    product_quanitity = product['data-product-quantity']
    product_instock_boolean = product['data-product-stock']
    product_listing_number = int(product['data-product-list-position'])
    secondary_soup = BeautifulSoup(requests.get(product_href).content, 'html.parser')
    pricing_data = json.loads(secondary_soup.find('div', {'data-module':"product"})['data-product-settings'])['pricing']

    for tr_ in secondary_soup.find_all('tr'):
        if tr_.find('th').text == 'Year':
            year = tr_.find('td').text
        if tr_.find('th').text == 'Pure Metal Type':
            metal_type = tr_.find('td').text
        if tr_.find('th').text == 'Pure Metal Content':
            metal_content = tr_.find('td').text
        if tr_.find('th').text == 'Fineness':
            finess = tr_.find('td').text
        # if tr_.find('th').text == 'Packaging':
        #    packaging_type = tr_.find('td').text

    df = df.append({
        'Product Name': product_name,
        'Website': 'Royal Mint',
        'Product Quanity Available': int(product_quanitity),
        'Propuct Instock': product_instock_boolean,
        'Product Listing number': int(product_listing_number),
        'Product Year': int(year),
        'Metal Type': metal_type,
        'Metal Content': metal_content,
        'Oz weight': float(metal_content.split()[0]),
        'Finess': float(finess),
        'Gold Value': float(metal_content.split()[0])*(float(finess)/1000)*gold_price,
        'Price per gram': pricing_data[0]['Price']/(float(metal_content.split()[0])*oz) if pricing_data else None,
        'Price 1': pricing_data[0]['Price'] if pricing_data else None,
        'Price 2': pricing_data[1]['Price'] if len(pricing_data) >= 2 and pricing_data else None,
        'Price 5': pricing_data[2]['Price'] if len(pricing_data) >= 3 and pricing_data else None,
        'Price 10': pricing_data[3]['Price'] if len(pricing_data) >= 4 and pricing_data else None,
        'Price 20': pricing_data[4]['Price'] if len(pricing_data) >= 5 and pricing_data else None,
        'Price 50': pricing_data[5]['Price'] if len(pricing_data) >= 6 and pricing_data else None,
        'Price 100': pricing_data[6]['Price'] if len(pricing_data) >= 7 and pricing_data else None,
        'Product link': product_href
    }, ignore_index=True)
    

In [165]:
df.sort_values(by=['Price per gram'], inplace=True)

royal_mint_full_list = None
product_name = None
product_href = None
product_href = None
product_quanitity = None
product_instock_boolean = None
product_listing_number = None
secondary_soup = None
pricing_data = None
year = None
metal_type = None
metal_content = None
finess = None

# Chards

In [167]:
chards = r'https://www.chards.co.uk/category/buy-coins/a/gold/in-stock/2000-2022'
chard_base_url = r'https://www.chards.co.uk'
chard_soup = BeautifulSoup(requests.get(chards).content, 'html.parser')
pages = int(chard_soup.find(class_="pager col-xs-12 text-center").find(class_='last').find('a')['data-page'])+2
chard_page_specified = chard_soup.find(class_="pager col-xs-12 text-center").find(class_='last').find('a')['href'].split('=')[0]
chard_soup = None

In [168]:
for page_number in range(1,pages):
    chard_new_web = f'{chard_base_url}{chard_page_specified}={page_number}'
    chard_new_soup = BeautifulSoup(requests.get(chard_new_web).content, 'html.parser')
    products = chard_new_soup.find(
        class_="products-grid col-xs-12 flex flex-direction-row flex-wrap"
        ).find_all('a', class_="col-xs-6 col-md-4 col-lg-3 grid-view product-item"
    )
    for product in products:
        product_href = product['href']
        product_soup = BeautifulSoup(requests.get(product_href).content, 'html.parser')
        product_name = product_soup.find(id="js-product-name").text
        
        table_titles = product_soup.find_all(class_='col-xs-6 col-sm-12 col-md-6 title')
        table_values = product_soup.find_all(class_='col-xs-6 col-sm-12 col-md-6 value')
        
        product_listing_number = int(table_values[0].text) if table_titles and table_titles[0].text == 'Product ID' else None
        product_instock_boolean = table_values[1].text if len(table_titles)>=2 and table_titles[1].text == 'Stock status' else None

        # 1,5,10,25,50,100
        pricing_data = product_soup.find(class_="table table-quantity-breaks")
        if not pricing_data:
            pricing_data_1 = float(product_soup.find(id='js-price')['data-price'])
        else:
            pricing_data = pricing_data.find_all('tr')
            pricing_data_1 = float(pricing_data[1].find(class_="product-price")['data-price'])
            

        value_text = table_values[2].text if len(table_titles)>=3 and table_titles[2].text == 'Intrinsic Value' else None
        gold_value = float(value_text.replace(',', '')[1:]) if value_text else None
        
        
        name_numbers = re.findall("[-+]?[.]?[\d]+(?:,\d\d\d)*[\.]?\d*(?:[eE][-+]?\d+)?", product_name)
        weight = [float(num) for num in name_numbers if float(num)<2000]
        weight = weight[0] if weight else None
        
        
        if product_name.lower().find('oz') and weight:
            relevant_weight = weight * oz
        elif not product_name.lower().find('oz') and product_name.lower().find('g ') and weight:
            relevant_weight = weight
        else:
            relevant_weight = None
            print('Unknown weight measurements found')
    
        df = df.append({
            'Product Name': product_name,
            'Website': 'Chards',
            'Propuct Instock': product_instock_boolean,
            'Product Listing number': int(product_listing_number),
            'Gold Value': gold_value,
            'Price per gram': relevant_weight*pricing_data_1/oz if pricing_data_1 and relevant_weight else None,
            'Price 1': pricing_data_1,
            'Price 2': None,
            'Price 5': float(pricing_data[2].find(class_="product-price")['data-price']) if pricing_data and len(pricing_data) >= 3 else None,
            'Price 10': float(pricing_data[3].find(class_="product-price")['data-price']) if pricing_data and len(pricing_data) >= 4 else None,
            'Price 20': float(pricing_data[4].find(class_="product-price")['data-price']) if pricing_data and len(pricing_data) >= 5 else None,
            'Price 50': float(pricing_data[5].find(class_="product-price")['data-price']) if pricing_data and len(pricing_data) >= 6 else None,
            'Price 100': float(pricing_data[6].find(class_="product-price")['data-price']) if pricing_data and len(pricing_data) >= 7 else None,
            'Product link': product_href
        }, ignore_index=True)
        
# chard_new_web = None, chard_new_soup = None, products = None
# product_href = None, product_soup = None, product_name = None
# table_titles = None, table_values = None
# product_listing_number = None, product_instock_boolean = None
# pricing_data = None, pricing_data_1 = None
# value_text = None, gold_value = None

Unknown weight measurements found
Unknown weight measurements found
Unknown weight measurements found
Unknown weight measurements found
Unknown weight measurements found
Unknown weight measurements found
Unknown weight measurements found
Unknown weight measurements found
Unknown weight measurements found
Unknown weight measurements found
Unknown weight measurements found
Unknown weight measurements found
Unknown weight measurements found
Unknown weight measurements found
Unknown weight measurements found
Unknown weight measurements found
Unknown weight measurements found
Unknown weight measurements found
Unknown weight measurements found
Unknown weight measurements found
Unknown weight measurements found
Unknown weight measurements found
Unknown weight measurements found
Unknown weight measurements found
Unknown weight measurements found
Unknown weight measurements found
Unknown weight measurements found
Unknown weight measurements found
Unknown weight measurements found
Unknown weight

In [166]:
# df.sort_values(by=['Price per gram'], inplace=True)
df

,Product Name,Product Quanity Available,Propuct Instock,Product Listing number,Product Year,Metal Type,Metal Content,Oz weight,Finess,Gold Value,Price per gram,Price 1,Price 2,Price 5,Price 10,Price 20,Price 50,Price 100,Product link,Website
112,Britannia 2022 1 oz Gold Bullion Ten Coin Tube,0,AwaitingStock,30,2022,Gold,10 x 1 Troy Oz,10.000,999.90,15068.493000,56.828363,16110.84105,16085.23242,16058.1174,16031.00238,16005.39375,15978.27873,15952.6701,https://www.royalmint.com/invest/bullion/bulli...,Royal Mint
73,Britannia 2022 1 oz Gold Bullion Ten Coin Tube,0,AwaitingStock,30,2022,Gold,10 x 1 Troy Oz,10.000,999.90,15068.493000,56.842322,16114.7982,16089.18328,16062.0616,16034.93992,16009.325,15982.20332,15956.5884,https://www.royalmint.com/invest/bullion/bulli...,Royal Mint
34,Britannia 2022 1 oz Gold Bullion Ten Coin Tube,0,AwaitingStock,30,2022,Gold,10 x 1 Troy Oz,10.000,999.90,15068.493000,56.849489,16116.83025,16091.2121,16064.087,16036.9619,16011.34375,15984.21865,15958.6005,https://www.royalmint.com/invest/bullion/bulli...,Royal Mint
88,The Double Sovereign 2022 Gold Bullion Coin,302,InStock,6,2022,Gold,0.47 Troy Oz,0.470,916.70,649.289443,57.075417,760.5014,759.439149,757.527096,755.969127,754.411157,752.286654,750.657868,https://www.royalmint.com/invest/bullion/bulli...,Royal Mint
82,The Double Sovereign 2022 Gold Bullion Coin,302,InStock,5,2022,Gold,0.47 Troy Oz,0.470,916.70,649.289443,57.075417,760.5014,759.439149,757.527096,755.969127,754.411157,752.286654,750.657868,https://www.royalmint.com/invest/bullion/bulli...,Royal Mint
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30,The Double Sovereign 2020 Gold Bullion Fifteen...,7,LowStock,26,2020,Gold,0.47 Troy Oz,0.470,916.67,649.268194,847.240778,11289.059743,11273.123747,11245.501354,11229.565357,11213.629361,11197.693365,11181.757369,https://www.royalmint.com/invest/bullion/bulli...,Royal Mint
116,The Sovereign 2022 Twenty Five Bullion Coin Tube,0,AwaitingStock,34,2022,Gold,0.235 Troy Oz,0.235,916.67,324.634097,1424.428569,9489.899232,9474.854162,9459.809092,9436.798985,9421.753915,9406.708845,9391.663775,https://www.royalmint.com/invest/bullion/bulli...,Royal Mint
77,The Sovereign 2022 Twenty Five Bullion Coin Tube,0,AwaitingStock,34,2022,Gold,0.235 Troy Oz,0.235,916.67,324.634097,1424.778437,9492.230144,9477.181379,9462.132613,9439.116855,9424.068089,9409.019324,9393.970558,https://www.royalmint.com/invest/bullion/bulli...,Royal Mint
38,The Sovereign 2022 Twenty Five Bullion Coin Tube,0,AwaitingStock,34,2022,Gold,0.235 Troy Oz,0.235,916.67,324.634097,1424.9581,9493.427099,9478.376436,9463.325773,9440.307112,9425.256449,9410.205786,9395.155123,https://www.royalmint.com/invest/bullion/bulli...,Royal Mint
